### Combining and Splitting Data in Spark

In this notebook we demonstrate how to combine and split Spark DataFrames. It is based on material supplied by Cloudera under their Cloudera Academic Partner program and *Spark: The Definitive Guide* book by Bill Chambers and Matei Zaharia. 

Topics
- Joining DataFrames
- Applying set operations to DataFrames
- Splitting a DataFrame 

#### Joining DataFrames

We will use the following datasets to demonstrate joins

In [0]:
scientists = spark.read.csv("/mnt/cis442f-data/duocar/raw/data_scientists/", header=True, inferSchema=True)
scientists.show()

offices = spark.read.csv("/mnt/cis442f-data/duocar/raw/offices/", header=True, inferSchema=True)
offices.show()

+-----------+----------+---------+---------+
employee_id|first_name|last_name|office_id|
+-----------+----------+---------+---------+
 63| Sophia| Laurent| 1|
 88| Thomas| Dubois| 1|
 107| Emmanuel| Auzenne| null|
 371| Martina| Ruiz| 5|
+-----------+----------+---------+---------+

+---------+-----------+---------+-------+
office_id|postal_code| city|country|
+---------+-----------+---------+-------+
 1| 93300| Paris| France|
 2| 13006|Marseille| France|
 3| 69002| Lyon| France|
 4| 31000| Toulouse| France|
 5| 58102| Fargo| USA|
+---------+-----------+---------+-------+

Use the `join` DataFrame method with different values of the `how` argument to perform various types of joins.

#### Inner join

In [0]:
# Use a join expression and the value `inner` to return only those rows 
# for which the join expression is true.  This gives us a list of data 
# scientists associated with an office and the corresponding office 
# information.

scientists.join(offices, scientists.office_id == offices.office_id, "inner") \
  .withColumnRenamed("employee_id", "emp_id") \
  .show()

+------+----------+---------+---------+---------+-----------+-----+-------+
emp_id|first_name|last_name|office_id|office_id|postal_code| city|country|
+------+----------+---------+---------+---------+-----------+-----+-------+
 88| Thomas| Dubois| 1| 1| 93300|Paris| France|
 63| Sophia| Laurent| 1| 1| 93300|Paris| France|
 371| Martina| Ruiz| 5| 5| 58102|Fargo| USA|
+------+----------+---------+---------+---------+-----------+-----+-------+

In [0]:
# Since the join key has the same name on both DataFrames, we can simplify
# the join to the following

scientists.join(offices, "office_id", "inner") \
  .withColumnRenamed("employee_id", "emp_id") \
  .show()

+---------+------+----------+---------+-----------+-----+-------+
office_id|emp_id|first_name|last_name|postal_code| city|country|
+---------+------+----------+---------+-----------+-----+-------+
 1| 88| Thomas| Dubois| 93300|Paris| France|
 1| 63| Sophia| Laurent| 93300|Paris| France|
 5| 371| Martina| Ruiz| 58102|Fargo| USA|
+---------+------+----------+---------+-----------+-----+-------+

In [0]:
# Since an inner join is the default, we can further simplify the join

scientists.join(offices, "office_id") \
  .withColumnRenamed("employee_id", "emp_id") \
  .show()

+---------+------+----------+---------+-----------+-----+-------+
office_id|emp_id|first_name|last_name|postal_code| city|country|
+---------+------+----------+---------+-----------+-----+-------+
 1| 88| Thomas| Dubois| 93300|Paris| France|
 1| 63| Sophia| Laurent| 93300|Paris| France|
 5| 371| Martina| Ruiz| 58102|Fargo| USA|
+---------+------+----------+---------+-----------+-----+-------+

#### Left outer join

Use the value `left` or `left_outer` to return every row in the left DataFrame with or without a matching row in the right DataFrame

In [0]:
# This gives us a list of data scientists with or without an office.

scientists \
  .join(offices, scientists.office_id == offices.office_id, "left_outer") \
  .withColumnRenamed("employee_id", "emp_id") \
  .show()

+------+----------+---------+---------+---------+-----------+-----+-------+
emp_id|first_name|last_name|office_id|office_id|postal_code| city|country|
+------+----------+---------+---------+---------+-----------+-----+-------+
 63| Sophia| Laurent| 1| 1| 93300|Paris| France|
 88| Thomas| Dubois| 1| 1| 93300|Paris| France|
 107| Emmanuel| Auzenne| null| null| null| null| null|
 371| Martina| Ruiz| 5| 5| 58102|Fargo| USA|
+------+----------+---------+---------+---------+-----------+-----+-------+

#### Right outer join

Use the value `right` or `right_outer` to return every row in the right DataFrame with or without a matching row in the left DataFrame

In [0]:
# This gives us a list of offices with or without a data scientist
# Note: The Paris office has two data scientists
scientists \
  .join(offices, scientists.office_id == offices.office_id, "right_outer") \
  .withColumnRenamed("employee_id", "emp_id") \
  .withColumnRenamed("postal_code", "p_code") \
  .show()

+------+----------+---------+---------+---------+------+---------+-------+
emp_id|first_name|last_name|office_id|office_id|p_code| city|country|
+------+----------+---------+---------+---------+------+---------+-------+
 88| Thomas| Dubois| 1| 1| 93300| Paris| France|
 63| Sophia| Laurent| 1| 1| 93300| Paris| France|
 null| null| null| null| 2| 13006|Marseille| France|
 null| null| null| null| 3| 69002| Lyon| France|
 null| null| null| null| 4| 31000| Toulouse| France|
 371| Martina| Ruiz| 5| 5| 58102| Fargo| USA|
+------+----------+---------+---------+---------+------+---------+-------+

#### Full outer join

Use the value `full`, `outer`, or `full_outer` to return the union of the left outer and right outer joins (with duplicates removed)

In [0]:
# This gives us a list of all data scientists whether or not they 
# have an office and all offices whether or not they have any data scientists.

scientists \
  .join(offices, scientists.office_id == offices.office_id, "full_outer") \
  .withColumnRenamed("employee_id", "emp_id") \
  .withColumnRenamed("postal_code", "p_code") \
  .show()


+------+----------+---------+---------+---------+------+---------+-------+
emp_id|first_name|last_name|office_id|office_id|p_code| city|country|
+------+----------+---------+---------+---------+------+---------+-------+
 107| Emmanuel| Auzenne| null| null| null| null| null|
 63| Sophia| Laurent| 1| 1| 93300| Paris| France|
 88| Thomas| Dubois| 1| 1| 93300| Paris| France|
 null| null| null| null| 2| 13006|Marseille| France|
 null| null| null| null| 3| 69002| Lyon| France|
 null| null| null| null| 4| 31000| Toulouse| France|
 371| Martina| Ruiz| 5| 5| 58102| Fargo| USA|
+------+----------+---------+---------+---------+------+---------+-------+

#### Left semi join

Use the value `left_semi` to return the rows in the left DataFrame that match rows in the right DataFrame.
Thus this sort of join does not include any values form the right DataFrame. We can think of left semi-join as a filter on the DataFrame.

In [0]:
# This gives us a list of data scientists associated with an office
scientists \
  .join(offices, scientists.office_id == offices.office_id, "left_semi") \
  .show()

+-----------+----------+---------+---------+
employee_id|first_name|last_name|office_id|
+-----------+----------+---------+---------+
 63| Sophia| Laurent| 1|
 88| Thomas| Dubois| 1|
 371| Martina| Ruiz| 5|
+-----------+----------+---------+---------+

#### Left anti join

Use the value `left_anti` to return the rows in the left DataFrame that do not match rows in the right DataFrame

Thus you can also think of the left anti join as a special type of filter

In [0]:
# This gives us a list of data scientists not associated with an office
scientists \
  .join(offices, scientists.office_id == offices.office_id, "left_anti") \
  .show() 

+-----------+----------+---------+---------+
employee_id|first_name|last_name|office_id|
+-----------+----------+---------+---------+
 107| Emmanuel| Auzenne| null|
+-----------+----------+---------+---------+

#### Cross join

Use the `crossJoin` DataFrame method to join every row in the left (scientists) DataFrame with every row in the right (offices) DataFrame. 

**Warning:** This can result in very big DataFrames!

**Note:**
- Columns with the same name are not renamed.
- This is called the *Cartesian product* of the two DataFrames.

In [0]:
# Use the `crossJoin` DataFrame method to join every row in the left 
# (scientists) DataFrame with every row in the right (offices) DataFrame
scientists.crossJoin(offices)\
  .withColumnRenamed("employee_id", "emp_id") \
  .withColumnRenamed("postal_code", "p_code") \
  .show()

+------+----------+---------+---------+---------+------+---------+-------+
emp_id|first_name|last_name|office_id|office_id|p_code| city|country|
+------+----------+---------+---------+---------+------+---------+-------+
 63| Sophia| Laurent| 1| 1| 93300| Paris| France|
 88| Thomas| Dubois| 1| 1| 93300| Paris| France|
 107| Emmanuel| Auzenne| null| 1| 93300| Paris| France|
 371| Martina| Ruiz| 5| 1| 93300| Paris| France|
 63| Sophia| Laurent| 1| 2| 13006|Marseille| France|
 88| Thomas| Dubois| 1| 2| 13006|Marseille| France|
 107| Emmanuel| Auzenne| null| 2| 13006|Marseille| France|
 371| Martina| Ruiz| 5| 2| 13006|Marseille| France|
 63| Sophia| Laurent| 1| 3| 69002| Lyon| France|
 88| Thomas| Dubois| 1| 3| 69002| Lyon| France|
 107| Emmanuel| Auzenne| null| 3| 69002| Lyon| France|
 371| Martina| Ruiz| 5| 3| 69002| Lyon| France|
 63| Sophia| Laurent| 1| 4| 31000| Toulouse| France|
 88| Thomas| Dubois| 1| 4| 31000| Toulouse| France|
 107| Emmanuel| Auzenne| null| 4| 31000| Toulouse| France|
 371| Martina| Ruiz| 5| 4| 31000| Toulouse| France|
 63| Sophia| Laurent| 1| 5| 58102| Fargo| USA|
 88| Thomas| Dubois| 1| 5| 58102| Fargo| USA|
 107| Emmanuel| Auzenne| null| 5| 58102| Fargo| USA|
 371| Martina| Ruiz| 5| 5| 58102| Fargo| USA|
+------+----------+---------+---------+---------+------+---------+-------+

#### Example: Joining the DuoCar data

Let us join the driver, rider, and review data with the ride data.

In [0]:
# Read the clean data
rides = spark.read.parquet("/mnt/cis442f-data/duocar/clean/rides/")
drivers = spark.read.parquet("/mnt/cis442f-data/duocar/clean/drivers/")
riders = spark.read.parquet("/mnt/cis442f-data/duocar/clean/riders/")
reviews = spark.read.parquet("/mnt/cis442f-data/duocar/clean/ride_reviews/")

In [0]:
# Since we want all the ride data, we will use a sequence of left outer joins
# Note that the id fields from three of the tables remain in the schema
joined = rides \
  .join(drivers, rides.driver_id == drivers.id, "left_outer") \
  .join(riders, rides.rider_id == riders.id, "left_outer") \
  .join(reviews, rides.id == reviews.ride_id, "left_outer")
joined.printSchema()

root
-- id: string (nullable = true)
-- driver_id: string (nullable = true)
-- rider_id: string (nullable = true)
-- date_time: timestamp (nullable = true)
-- utc_offset: integer (nullable = true)
-- service: string (nullable = true)
-- origin_lat: decimal(9,6) (nullable = true)
-- origin_lon: decimal(9,6) (nullable = true)
-- dest_lat: decimal(9,6) (nullable = true)
-- dest_lon: decimal(9,6) (nullable = true)
-- distance: integer (nullable = true)
-- duration: integer (nullable = true)
-- cancelled: boolean (nullable = true)
-- star_rating: integer (nullable = true)
-- id: string (nullable = true)
-- birth_date: date (nullable = true)
-- start_date: date (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- sex: string (nullable = true)
-- ethnicity: string (nullable = true)
-- student: boolean (nullable = true)
-- home_block: string (nullable = true)
-- home_lat: decimal(9,6) (nullable = true)
-- home_lon: decimal(9,6) (nullable = true)
-- vehicle_make: string (nullable = true)
-- vehicle_model: string (nullable = true)
-- vehicle_year: integer (nullable = true)
-- vehicle_color: string (nullable = true)
-- vehicle_grand: boolean (nullable = true)
-- vehicle_noir: boolean (nullable = true)
-- vehicle_elite: boolean (nullable = true)
-- rides: integer (nullable = true)
-- stars: integer (nullable = true)
-- id: string (nullable = true)
-- birth_date: date (nullable = true)
-- start_date: date (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- sex: string (nullable = true)
-- ethnicity: string (nullable = true)
-- student: boolean (nullable = true)
-- home_block: string (nullable = true)
-- home_lat: decimal(9,6) (nullable = true)
-- home_lon: decimal(9,6) (nullable = true)
-- work_lat: decimal(9,6) (nullable = true)
-- work_lon: decimal(9,6) (nullable = true)
-- ride_id: string (nullable = true)
-- review: string (nullable = true)

In [0]:
# We can disambiguate the fields with the same names as follows
joined.select(rides.id.alias("rides_id2"), joined.driver_id, drivers.id.alias("driver_id2"), \
              joined.rider_id, riders.id.alias("rider_id2")).show(5)

+----------+------------+------------+------------+------------+
 rides_id2| driver_id| driver_id2| rider_id| rider_id2|
+----------+------------+------------+------------+------------+
0000000001|220200000214|220200000214|220200000084|220200000084|
0000000002|220200000107|220200000107|220200000462|220200000462|
0000000003|220200000214|220200000214|220200000489|220200000489|
0000000004|220200000067|220200000067|220200000057|220200000057|
0000000005|220200000067|220200000067|220200000012|220200000012|
+----------+------------+------------+------------+------------+
only showing top 5 rows

In [0]:
# We could change column names before the join but we can drop duplicate columns
# by refering to parent DataFrames afterwards as well
joined = rides \
  .join(drivers, rides.driver_id == drivers.id, "left_outer") \
  .join(riders, rides.rider_id == riders.id, "left_outer") \
  .join(reviews, rides.id == reviews.ride_id, "left_outer") \
  .drop(riders.id)\
  .drop(drivers.id)\
  .drop(reviews.ride_id)
  
# In principle we can also create a new DataFrame with unabiguous column names as well  
joined.select(drivers["home_block"].alias("driver_home_block")).printSchema()

# withColumnRename does not work as it only takes a string for the column name and will
# not resolve a column name including the parent DataFrame

root
-- driver_home_block: string (nullable = true)

#### Applying set operations to DataFrames

Spark SQL provides the following DataFrame methods that implement set operations
- `union`
- `intersect`
- `subtract`

In [0]:
# Use the `union` method to get the union of rows in two DataFrames with similar schema
driver_names = drivers.select("first_name")
print(driver_names.count())

rider_names = riders.select("first_name")
print(rider_names.count())

names_union = driver_names.union(rider_names).orderBy("first_name")
print (names_union.count())
names_union.show()

563
1723
2286
+-----------+
 first_name|
+-----------+
 A D|
 Aaron|
 Aaron|
 Aaron|
 Aaron|
 Aaron|
 Aaron|
 Aaron|
 Aaron|
Aaron James|
 Abby|
 Abigail|
 Abigail|
 Abraham|
 Abram|
 Acacia|
 Adam|
 Adam|
 Adam|
 Adam|
+-----------+
only showing top 20 rows

In [0]:
# Note that `union` does not remove duplicates.  Use the `distinct` method to remove duplicates
names_distinct = names_union.distinct()
print(names_distinct.count())
names_distinct.show()

911
+----------+
first_name|
+----------+
 James|
 Brooke|
 Matthew|
 Jordan|
 Keegan|
 Dale|
 Logan|
 Minh|
 Eric|
 Raymond|
 Dillon|
 C|
 Sean|
 Leonard|
 Patrick|
 Adam|
 Alexander|
 Collin|
 Zachary|
 Douglas|
+----------+
only showing top 20 rows

In [0]:
# Use the `intersect` method to return rows that exist in both DataFrames
name_intersect = driver_names.intersect(rider_names).orderBy("first_name")
print(name_intersect.count())
name_intersect.show()

182
+----------+
first_name|
+----------+
 Aaron|
 Abigail|
 Adam|
 Adrian|
 Aimee|
 Alec|
 Alex|
 Alexa|
 Alexander|
 Alexandra|
 Alexis|
 Allison|
 Amanda|
 Andrew|
 Anna|
 Anthony|
 Ashlee|
 Ashley|
 Austin|
 Avery|
+----------+
only showing top 20 rows

In [0]:
# Use the `subtract` method to return rows in the left DataFrame that do not exist in the right DataFrame
names_subtract = driver_names.subtract(rider_names).orderBy("first_name")
print(names_subtract.count())
names_subtract.show()


149
+----------+
first_name|
+----------+
 A D|
 Abram|
 Aerial|
 Airik|
 Allan|
 Alvina|
 Bobby|
 Braxton|
 Brendan|
 Brendon|
 Brock|
 C|
 Cahl|
 Callan|
 Calvin|
 Candace|
 Caris|
 Carson|
 Celeste|
 Chander|
+----------+
only showing top 20 rows

#### Splitting a DataFrame

Use the `randomSplit` DataFrame method to split a DataFrame into random subsets

**Note:** 
- The proportions must be doubles.
- We will use this functionality to create train, validation, and test datasets for machine learning pipelines.

In [0]:
# Use the `randomSplit` DataFrame method to split a DataFrame into random
# subsets. Use the `seed` argument to ensure replicability
riders.count()
(train, validate, test) = riders.randomSplit([0.6, 0.2, 0.2], seed=42)
print (train.count())
print(validate.count())
print(test.count())

1072
354
297

In [0]:
# If the proportions do not add up to one, then Spark will normalize the values
(train, validate, test) = riders.randomSplit([60.0, 20.0, 20.0], seed=42)
print (train.count())
print(validate.count())
print(test.count())

1072
354
297

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")

#### Exercises

(1) Create a DataFrame with all combinations of vehicle make and vehicle year (regardless of whether the combination is observed in the data). Use the drivers dataset `drivers = spark.read.parquet("/mnt/cis442f-data/duocar/clean/drivers/")`

(2) Join the demographic data (on block_group) and weather data (on date) with the joined rides data. Demographic and weather data are in the 'raw' directory. Use the joined dataset `joined = spark.read.parquet("/mnt/cis442f-data/duocar/joined/")`

(3) Are there any drivers who have not provided a ride? Use the drivers and rides datasets `rides = spark.read.parquet("/mnt/cis442f-data/duocar/clean/rides/")`


#### References

join, crossJoin, union, intersect, subtract, and randomSplit are all methods of the [Spark DataFrame class](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html#pyspark.sql.DataFrame)